# Date Night Movie

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


In this assignment we are going to use pandas to figure out - What's the best **date-night movie**?

This assignment is going to use
- Joining
- Groupby
- Sorting


In [1]:
import os
import pandas as pd

##### Read in the movie data: `pd.read_table`

In [2]:
def get_movie_data():
    
    unames = ['user_id','gender','age','occupation','zip']
    users = pd.read_table(os.path.join('../data','users.dat'), 
                          sep='::', header=None, names=unames, encoding="ISO-8859-1")
    
    rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
    ratings = pd.read_table(os.path.join('../data', 'ratings.dat'), 
                            sep='::', header=None, names=rnames , encoding="ISO-8859-1")
    
    mnames = ['movie_id', 'title','genres']
    movies = pd.read_table(os.path.join('../data', 'movies.dat'), 
                           sep='::', header=None, names=mnames , encoding="ISO-8859-1")

    return users, ratings, movies

In [3]:
users, ratings, movies = get_movie_data()

C:\Users\p_bud\AppData\Local\Temp/ipykernel_2944/1537425519.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_table(os.path.join('../data','users.dat'),
C:\Users\p_bud\AppData\Local\Temp/ipykernel_2944/1537425519.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_table(os.path.join('../data', 'ratings.dat'),
C:\Users\p_bud\AppData\Local\Temp/ipykernel_2944/1537425519.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid t

In [4]:
print(users.head())

   user_id gender  age  occupation    zip
0        1      F    1          10  48067
1        2      M   56          16  70072
2        3      M   25          15  55117
3        4      M   45           7  02460
4        5      M   25          20  55455


In [5]:
print(ratings.head())

   user_id  movie_id  rating  timestamp
0        1      1193       5  978300760
1        1       661       3  978302109
2        1       914       3  978301968
3        1      3408       4  978300275
4        1      2355       5  978824291


In [6]:
print(movies.head())

   movie_id                               title                        genres
0         1                    Toy Story (1995)   Animation|Children's|Comedy
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy
2         3             Grumpier Old Men (1995)                Comedy|Romance
3         4            Waiting to Exhale (1995)                  Comedy|Drama
4         5  Father of the Bride Part II (1995)                        Comedy


##### Clean up the `movies`

- Get the `year`
- Shorten the `title`


In [7]:
tmp = movies.title.str.extract('(.*) \(([0-9]+)\)')
tmp.apply(lambda x:x[0] if len(x) > 0 else None)
tmp.apply(lambda x: x[0][:40] if len(x) > 0 else None)

0    Toy Story
1         1995
dtype: object

In [8]:
movies['year'] = tmp[1]
movies['short_title'] = tmp[0]

In [9]:
print(movies.head())

   movie_id                               title                        genres  \
0         1                    Toy Story (1995)   Animation|Children's|Comedy   
1         2                      Jumanji (1995)  Adventure|Children's|Fantasy   
2         3             Grumpier Old Men (1995)                Comedy|Romance   
3         4            Waiting to Exhale (1995)                  Comedy|Drama   
4         5  Father of the Bride Part II (1995)                        Comedy   

   year                  short_title  
0  1995                    Toy Story  
1  1995                      Jumanji  
2  1995             Grumpier Old Men  
3  1995            Waiting to Exhale  
4  1995  Father of the Bride Part II  


##### Join the tables with `pd.merge` (20 pts)

* For this task I will use merge() function to merge all the three tables that is users , ratings and movies. I will merge 
  users and ratings table first and then merge the movies table. 

In [10]:
#checking the shape of the tables
users.shape , ratings.shape , movies.shape

((6040, 5), (1000209, 4), (3883, 5))

* Since user_id is common variable between users and ratings I will perform inner merge using user_id. 

In [11]:
user_rating_table_merge = pd.merge(users , ratings, on = 'user_id')

In [12]:
user_rating_table_merge.shape

(1000209, 8)

* Now merge the movies table with the above created table. But before that check the common variables in both tables. 
* Below we see that movie_id is common to both tables i.e  user_rating_table_merge and movies , so I perform inner merge on movie_id

In [13]:
user_rating_table_merge.columns

Index(['user_id', 'gender', 'age', 'occupation', 'zip', 'movie_id', 'rating',
       'timestamp'],
      dtype='object')

In [14]:
movies.columns

Index(['movie_id', 'title', 'genres', 'year', 'short_title'], dtype='object')

In [15]:
user_rating_movies_table_merge = pd.merge(user_rating_table_merge , movies, on = 'movie_id')

In [16]:
user_rating_movies_table_merge.shape

(1000209, 12)

#### Lets check for null values by using isnull() and summing them to get column wise information.

In [17]:
user_rating_movies_table_merge.isnull().sum()

user_id        0
gender         0
age            0
occupation     0
zip            0
movie_id       0
rating         0
timestamp      0
title          0
genres         0
year           0
short_title    0
dtype: int64

#### Drop Duplicates and check for the shape of the data. 

In [18]:
#droping duplicates
user_rating_movies_table_merge.drop_duplicates().shape

(1000209, 12)

#### There are no null values and now the data is clean to perform the below tasks

##### What's the highest rated movie? (20 pts))

In [19]:
user_rating_movies_table_merge.head()

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title
0,1,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
1,2,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
2,12,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
3,15,M,25,7,22903,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest
4,17,M,50,1,95350,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama,1975,One Flew Over the Cuckoo's Nest


#### checking for rating range

In [20]:
user_rating_movies_table_merge.rating.unique()

array([5, 4, 3, 2, 1], dtype=int64)

#### For this task , I will group by movie_id and title

In [21]:
grouped_data = user_rating_movies_table_merge.groupby(['movie_id' ,'title'])

In [22]:
grouped_data

#### I am counting the user ID as each movie is associated with user_id and rating so when we group by movie_id we also need to count how many users rated for that movie. That gives a total votes for that rating.

    * https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html

In [23]:
grouped_data = grouped_data.agg({'user_id': 'count','rating':'mean'}).reset_index()

In [ ]:
grouped_data = grouped_data.rename(columns = {'user_id' : 'total_votes_count'})

In [24]:
grouped_data

,movie_id,title,total_votes_count,rating
0,1,Toy Story (1995),2077,4.146846
1,2,Jumanji (1995),701,3.201141
2,3,Grumpier Old Men (1995),478,3.016736
3,4,Waiting to Exhale (1995),170,2.729412
4,5,Father of the Bride Part II (1995),296,3.006757
...,...,...,...,...
3701,3948,Meet the Parents (2000),862,3.635731
3702,3949,Requiem for a Dream (2000),304,4.115132
3703,3950,Tigerland (2000),54,3.666667
3704,3951,Two Family House (2000),40,3.900000


### The highest rated movie can be either by ratings alone or by ratings and the no of votes given to the movie which is nothing but the total_users.
* But only by rating is not a good idea to get the highest rated movie. This is because if a movie has 5 rating and there are only 10 votes doesnt mean that the movie is the best. For this reason I calculated the minimum votes a movie should have. For this task I used quartile method and tried to get the votes between the 3rd and 1st quartile. I took this idea from the notes and got the minvotes from the data. 

    * https://github.com/UNCG-CSE/CSC-605_Fall_2022/blob/main/Class_Resources/Lecture_06/Statistics/01_Stats_Basics.ipynb
    * https://www.statology.org/and-operator-in-pandas/

In [25]:
mean_rating = grouped_data.rating.mean()
print(mean_rating)

3.238892177910887


In [26]:
minvotes = grouped_data["total_votes_count"].quantile(0.75) - grouped_data["total_votes_count"].quantile(0.25)

In [27]:
minvotes

317.0

Now create a new dataframe with rating being greater than mean and total_votes for a movie being greater than minvotes. Then sort the ratings column from highest to lowest and get the first movie from the dataset. This is nothing but highly rated movie.

In [28]:
by_users_rating = grouped_data[(grouped_data.rating > mean_rating) & (grouped_data.total_votes_count > minvotes)]

In [29]:
high_rated_movie = by_users_rating.sort_values(by= ['rating' ,'total_votes_count'] , ascending = False)

In [30]:
high_rated_movie.head(1)

,movie_id,title,total_votes_count,rating
1839,2019,Seven Samurai (The Magnificent Seven) (Shichin...,628,4.56051


###### What is a good rated movie for date night? (60 pts)

- Hint - highly rated movie by 
    - both partners (might be the same gender or not),
    - based on genre preferences,
    - age group can also be combined

### Trying to understand the unique genres and trying to filter genres with romance string in it as we are looking for date night movies.

    *https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html

In [31]:
genre = user_rating_movies_table_merge.genres.unique()
print(genre)

['Drama' "Animation|Children's|Musical" 'Musical|Romance'
 "Animation|Children's|Comedy" 'Action|Adventure|Comedy|Romance'
 'Action|Adventure|Drama' 'Comedy|Drama'
 "Adventure|Children's|Drama|Musical" 'Musical' 'Comedy'
 "Animation|Children's" 'Comedy|Fantasy' 'Animation' 'Comedy|Sci-Fi'
 'Drama|War' 'Romance' "Animation|Children's|Musical|Romance"
 "Children's|Drama|Fantasy|Sci-Fi" 'Drama|Romance'
 'Animation|Comedy|Thriller'
 "Adventure|Animation|Children's|Comedy|Musical"
 "Animation|Children's|Comedy|Musical" 'Thriller' 'Action|Crime|Romance'
 'Action|Adventure|Fantasy|Sci-Fi' "Children's|Comedy|Musical"
 'Action|Drama|War' "Children's|Drama" 'Crime|Drama|Thriller'
 'Action|Crime|Drama' 'Action|Adventure|Mystery' 'Crime|Drama'
 'Action|Adventure|Sci-Fi|Thriller' 'Action|Adventure|Romance|Sci-Fi|War'
 'Action|Thriller' 'Action|Drama' 'Comedy|Drama|Western'
 'Action|Adventure|Crime' 'Action|Crime|Mystery|Thriller'
 'Comedy|Drama|Romance' 'Comedy|Drama|War' 'Drama|Sci-Fi'
 'Action|Dr

#### Filtering the data with genre as romance in it and am trying to filter the age greater than 30 who is interested in watching movies released after 1997.  I converted the year to int as I was getting an error which said I am comparing string with int so after using astype.int() I was able to convert that column to int 

In [32]:
romantic_movie_data = user_rating_movies_table_merge[(user_rating_movies_table_merge.genres.str.contains("Romance")) & (user_rating_movies_table_merge.age>30)
                                       & ((user_rating_movies_table_merge.year.astype(int) > 1997))]
romantic_movie_data

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title
28160,45,F,45,16,94110,2340,2,977988826,Meet Joe Black (1998),Romance,1998,Meet Joe Black
28162,137,F,45,6,78758,2340,3,1000869643,Meet Joe Black (1998),Romance,1998,Meet Joe Black
28167,201,F,35,2,55117,2340,5,976947374,Meet Joe Black (1998),Romance,1998,Meet Joe Black
28172,326,M,50,11,25302,2340,3,980988524,Meet Joe Black (1998),Romance,1998,Meet Joe Black
28174,343,F,35,3,55127,2340,4,1040569658,Meet Joe Black (1998),Romance,1998,Meet Joe Black
...,...,...,...,...,...,...,...,...,...,...,...,...
999924,5039,F,35,4,97068,1770,2,962511522,B. Monkey (1998),Romance|Thriller,1998,B. Monkey
999927,5398,M,56,17,71301,1770,4,960300359,B. Monkey (1998),Romance|Thriller,1998,B. Monkey
999929,5413,M,35,1,59801,1770,4,960915308,B. Monkey (1998),Romance|Thriller,1998,B. Monkey
999930,5599,M,35,7,90020,1770,3,959213788,B. Monkey (1998),Romance|Thriller,1998,B. Monkey


In [33]:
grouped_data_romance = romantic_movie_data.groupby(['movie_id' ,'title' , 'genres' , 'gender' ,'age'])

In [34]:
grouped_data_romance

In [35]:
grouped_data_romance = grouped_data_romance.agg({'user_id': 'count','rating':'mean'}).reset_index()

grouped_data_romance = grouped_data_romance.rename(columns = {'user_id' : 'total_votes_count'})

#### Below is the dataframe with genre that contains romance keyword and age > 30 and movies after 1999 

In [36]:
grouped_data_romance

,movie_id,title,genres,gender,age,total_votes_count,rating
0,1680,Sliding Doors (1998),Drama|Romance,F,35,38,3.736842
1,1680,Sliding Doors (1998),Drama|Romance,F,45,15,3.533333
2,1680,Sliding Doors (1998),Drama|Romance,F,50,13,3.230769
3,1680,Sliding Doors (1998),Drama|Romance,F,56,4,4.500000
4,1680,Sliding Doors (1998),Drama|Romance,M,35,35,3.914286
...,...,...,...,...,...,...,...
677,3909,Woman on Top (2000),Comedy|Romance,F,56,1,3.000000
678,3909,Woman on Top (2000),Comedy|Romance,M,35,7,3.285714
679,3909,Woman on Top (2000),Comedy|Romance,M,45,3,2.333333
680,3909,Woman on Top (2000),Comedy|Romance,M,50,3,2.666667


In [37]:
mean_rating_romanctic_data = grouped_data_romance.rating.mean()
print(mean_rating_romanctic_data)

3.306161374496498


In [38]:
minvotes_romantic = grouped_data_romance["total_votes_count"].quantile(0.75) - grouped_data_romance["total_votes_count"].quantile(0.25)
print(minvotes_romantic)

10.0


In [39]:
high_rated_movie_by_genre_age_year = grouped_data_romance[(grouped_data_romance.rating > mean_rating_romanctic_data) & (grouped_data_romance.total_votes_count > minvotes_romantic) ]

In [40]:
high_rated_movie_by_genre_age_year = high_rated_movie_by_genre_age_year.sort_values(by= ['rating' ,'total_votes_count'] , ascending = False)

### high rated movie by genre romance ,  age > 30 and movie year > 1997

In [41]:
high_rated_movie_by_genre_age_year.head(1)

,movie_id,title,genres,gender,age,total_votes_count,rating
404,2692,Run Lola Run (Lola rennt) (1998),Action|Crime|Romance,F,45,19,4.421053


#### Another way of choosing the genre could be by most watched movie by youngters so lets see how we can calculate that 

In [42]:
genre = user_rating_movies_table_merge.genres.value_counts()
print(genre)

Comedy                       116883
Drama                        111423
Comedy|Romance                42712
Comedy|Drama                  42245
Drama|Romance                 29170
                              ...  
Drama|Romance|Western            29
Children's|Fantasy               27
Comedy|Film-Noir|Thriller         5
Film-Noir|Horror                  2
Fantasy                           1
Name: genres, Length: 301, dtype: int64


#### From the above we see that genre "Comedy|Romance" is most liked or watched. We calculated from value_counts which gives the total sum of  each genre, so we can use this information to filter the genre

In [43]:
age = user_rating_movies_table_merge.age.value_counts()
print(age)

25    395556
35    199003
18    183536
45     83633
50     72490
56     38780
1      27211
Name: age, dtype: int64


### From above we see that people with age = 25 watch movies frequently and now lets see which is the highest rated movie based on the above collected information for the young age group

In [44]:
romantic_movie_data_by_genre_age  = user_rating_movies_table_merge[(user_rating_movies_table_merge["genres"] == "Comedy|Romance") & (user_rating_movies_table_merge["age"] == 25) & (user_rating_movies_table_merge["year"].astype(int) > 1993)]
                                
romantic_movie_data_by_genre_age

,user_id,gender,age,occupation,zip,movie_id,rating,timestamp,title,genres,year,short_title
144107,8,M,25,12,11413,2396,5,978229524,Shakespeare in Love (1998),Comedy|Romance,1998,Shakespeare in Love
144109,11,F,25,1,04093,2396,2,978902561,Shakespeare in Love (1998),Comedy|Romance,1998,Shakespeare in Love
144111,15,M,25,7,22903,2396,4,978196817,Shakespeare in Love (1998),Comedy|Romance,1998,Shakespeare in Love
144115,24,F,25,7,10023,2396,4,978134987,Shakespeare in Love (1998),Comedy|Romance,1998,Shakespeare in Love
144116,28,F,25,1,14607,2396,5,978125846,Shakespeare in Love (1998),Comedy|Romance,1998,Shakespeare in Love
...,...,...,...,...,...,...,...,...,...,...,...,...
999628,2344,M,25,12,02139,1878,3,974414834,Woo (1998),Comedy|Romance,1998,Woo
999629,2378,M,25,3,75150,1878,4,974386496,Woo (1998),Comedy|Romance,1998,Woo
999630,2645,M,25,7,60657,1878,1,973559598,Woo (1998),Comedy|Romance,1998,Woo
999631,3067,F,25,0,02148,1878,2,996607212,Woo (1998),Comedy|Romance,1998,Woo


In [45]:
grouped_data_romanceby_genre_age = romantic_movie_data_by_genre_age.groupby(['movie_id' ,'title' , 'genres' , 'gender' ,'age'])

In [46]:
grouped_data_romanceby_genre_age

#### calculate the mean and minvotes as above

In [47]:
grouped_data_romanceby_genre_age = grouped_data_romanceby_genre_age.agg({'user_id': 'count','rating':'mean'}).reset_index()

grouped_data_romanceby_genre_age = grouped_data_romanceby_genre_age.rename(columns = {'user_id' : 'total_votes_count'})

In [48]:
mean_rating_romanctic_data_by_genre_age = grouped_data_romance.rating.mean()
print(mean_rating_romanctic_data_by_genre_age)

3.306161374496498


In [49]:
minvotes_romantic_by_genre_age = grouped_data_romance["total_votes_count"].quantile(0.75) - grouped_data_romance["total_votes_count"].quantile(0.25)
print(minvotes_romantic_by_genre_age)

10.0


In [50]:
high_rated_movie_by_genre_age = grouped_data_romanceby_genre_age[(grouped_data_romanceby_genre_age.rating> mean_rating_romanctic_data_by_genre_age) & (grouped_data_romanceby_genre_age.total_votes_count > minvotes_romantic_by_genre_age) ]

In [51]:
high_rated_movie_by_genre_age = high_rated_movie_by_genre_age.sort_values(by= ['rating' ,'total_votes_count'] , ascending = False).head(1)

In [52]:
high_rated_movie_by_genre_age

,movie_id,title,genres,gender,age,total_votes_count,rating
114,2396,Shakespeare in Love (1998),Comedy|Romance,F,25,276,4.206522
